# Part III

## Utilization
Analyze the capacity utilization of your solution from Part II. Calculate the % of storage capacity utilized at
each DC on each day, and calculate the % of truck capacity utilized at the start of each trip. Based on this
analysis, which capacity (storage or truck) seems to be the bottleneck?

In [1]:
using JuMP
using Gurobi
import Random


#------
# DATA
#------

### SETS
vehicles = [1 2 3];
K = length(vehicles);
DCs = [ 1 2 3 4 5 6];
I = length(DCs);
days = [1 2 3 4 5 6 7 8]
T = length(days); #number of time periods

Capdc = [999999 502 488 220 486 742]; #inventory holding capacity at every node including depot
Captruck = 900; #capacity of each truck
h = 0.025; # inventory holding cost at customers
Inv_begin = [0 98.8 153.9 42.4 23.4 85.7]; # inventory at the nodes at the beginning of the period

shippingCost = [0 140 434 389 419 125
    140 0 300 455 400 97
    434 300 0 609 417 330
    389 455 609 0 256 358
    419 400 417 256 0 316
    125 97 330 358 316 0
]; # transporation cost including from depot (matrix flipped from original data)

demand = [0 0 0 0 0 0 0 0 
    75.9 75.9 75.9 75.9 75.9 85.7 85.7 75.9
    62.9 94.8 94.8 94.8 94.8 68.65 36.75 62.9
    67.2 67.2 67.2 67.2 67.2 67.15 67.15 67.2
    102.3 156.0 156.0 156.0 156.0 119.05 65.35 102.3
    107.4 130.1 130.1 130.1 130.1 65.15 42.45 107.4
]; #demand at all customers (rows) for every time period(columns)





demand_std = [0 0 0 0 0 0 0 0
    11.87434209	11.87434209	11.87434209	11.87434209	11.87434209	14.68843082	14.68843082	11.87434209
    10.90871211	13.89244399	13.89244399	13.89244399	13.89244399	14.14213562	11.22497216	10.90871211
    11.26942767	11.26942767	11.26942767	11.26942767	11.26942767	12.86468033	12.86468033	11.26942767
    13.19090596	16.03121954	16.03121954	16.03121954	16.03121954	15.04991694	11.97914855	13.19090596
    13.03840481	14	        14	        14	        14	        11.0792599	9.836157786	13.03840481
];

Random.seed!(0);
actual_demand = zeros(10000,I-1,T)
for r = 1:10000
    actual_demand[r,:,:] = demand[2:end,:] + (randn(I-1,T).* demand_std[2:end,:])
end

za = 1.7263
model = Model(Gurobi.Optimizer);
@variable(model, x[i=1:I,j=1:I,k=1:K,t=1:T], Bin); #vehicle k goes from i to j in period t
@variable(model, y[i=1:I,k=1:K,t=1:T], Bin); #vehicle k visits node i in period t
@variable(model, z[i=2:I,k=1:K,t=1:T] >= 0); #load carried by vehicle k when arriving at node i in period t
@variable(model, p[i=1:I,t=1:T] >=0); # inventory at node i in time period t


@variable(model, q[i=2:I,k=1:K,t=1:T] >=0); # Quantity delivered to customer i by vehicle k at start of period t

@objective(model, Min,sum(h*p[i,t] for i in 2:I, t in 1:T) + sum(shippingCost[i,j]*x[i,j,k,t] for i in 1:I,j in 1:I,k in 1:K,t in 1:T));


#_______________ pure VRP ____________ #
# Constraint 4: A customer got visit by not more than one truck in one period:
@constraint(model, [i=2:I,t=1:T], sum(y[i,k,t] for k in 1:K) <= 1);

# Constraint 5: Vehicle capacity
@constraint(model, [k=1:K,t=1:T], sum(demand[i,t]*y[i,k,t] for i in 1:I) <= Captruck);
# Constraint 5_1 : Inflow: if vehicle k visits node i, inflow of vehicle k should be 1, 0, otherwise
#                  Outflow: if vehicle k visits node i, outflow of vehicle k should be 1, 0, otherwise
#part 1
@constraint(model, [i=1:I,k=1:K,t=1:T], sum(x[i,j,k,t] for j in 1:I) == sum(x[j,i,k,t] for j in 1:I));
#part 2
@constraint(model, [i=1:I,k=1:K,t=1:T], sum(x[i,j,k,t] for j in 1:I) == y[i,k,t]);

# Constraint 6: Subtour elimination constraint : load should reduce while going from i to j
@constraint(model, [i=2:I,j=2:I,k=1:K,t=1:T], z[i,k,t] - z[j,k,t] >= demand[i,t] - ((1-x[i,j,k,t]) * sum(demand[a,t] for a in 1:I) ));


#_______________ Inventory optimization
# Constraint 1: Flow balance constraints at depot in each period t:
#@constraint(model, [t=1:T-1], p[1,t] == p[1,t+1] + sum(q[i,k,t+1] for i in 2:I, k in 1:K));
# Constraint 2: Flow balance constraints at each customer i in each period t:
@constraint(model, [i=2:I,t=1:T], (t == 1 ? Inv_begin[i] : p[i,t-1]) + sum(q[i,k,t] for k in 1:K) == p[i,t] + demand[i,t] + za*demand_std[i,t])
# Constraint 3: Inventory capacity constraints at each customer i in each period t:
@constraint(model, [i=2:I,t=1:T], (t == 1 ? Inv_begin[i] : p[i,t-1]) + sum(q[i,k,t] for k in 1:K) <= Capdc[i]);


#_______________ Linking both parts
# Constraint 7: Linking the inventory part of the model to the VRP part:
    # Constraint 7a: Amount delivered to a customer cannot exceed the holding capacity of the customer
@constraint(model, [i=2:I,k=1:K,t=1:T], q[i,k,t] <= Capdc[i]*y[i,k,t]);
    # Constraint 7b: Inventory delivered to a customer cannot exceed the truck capacity
@constraint(model, [k=1:K,t=1:T], sum(q[i,k,t] for i in 2:I) <= Captruck*y[1,k,t]);

#-------
# SOLVE
#-------

optimize!(model);
println();
if termination_status(model) == MOI.OPTIMAL
    println("Optimal objective value = $(objective_value(model))")
    println("   ")
    println("----------------------")
    println("   ")
end

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-18
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1176 rows, 1296 columns and 4982 nonzeros
Model fingerprint: 0x5bcb849b
Variable types: 288 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+02]
Presolve removed 185 rows and 186 columns
Presolve time: 0.01s
Presolved: 991 rows, 1110 columns, 4404 nonzeros
Variable types: 273 continuous, 837 integer (837 binary)
Found heuristic solution: objective 11021.855219

Root relaxation: objective 2.727492e+03, 442 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0

### Storage Utilization

In [2]:
storage_utilisation = zeros(I-1, T)
inv = Inv_begin[2:end]
for t=1:T
    for i=2:I
        inv = value(p[i,t])
        storage_utilisation[i-1,t] += round(inv/Capdc[i],digits=2)
    end
end 
storage_utilisation

5×8 Matrix{Float64}:
 0.0   0.81  0.62  0.42  0.63  0.41  0.19  0.0
 0.83  0.59  0.35  0.1   0.47  0.28  0.17  0.0
 0.39  0.0   0.61  0.21  0.41  0.0   0.39  0.0
 0.48  0.1   0.5   0.12  0.3   0.0   0.26  0.0
 0.0   0.33  0.53  0.32  0.11  0.0   0.18  0.0

## Truck Utilization

In [3]:
Truck_utilisation = zeros(Float64, 3, 8)
for t=1:T
    for k = 1:K
        for i = 2:I
            Truck_utilisation[k,t] += value(q[i,k,t])
        end
        Truck_utilisation[k,t] = round(Truck_utilisation[k,t]/Captruck,digits=1)
    end
end
Truck_utilisation

3×8 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.6  0.0
 0.9  1.0  1.0  0.0  1.0  0.0  0.0  0.0

## 2. Consider three alternative capacity scenarios: 

In [4]:
Random.seed!(0);
actual_demand = zeros(10000,I-1,T)
for r = 1:10000
    actual_demand[r,:,:] = demand[2:end,:] + (randn(I-1,T).* demand_std[2:end,:])
end

function optimize_vrpinv(za,truck_factor,dc_factor)
    Captruck = 900*truck_factor; #capacity of each truck
    Capdc = [999999 502 488 220 486 742].*dc_factor; #inventory holding capacity at every node including depot

    model = Model(Gurobi.Optimizer);

    @variable(model, x[i=1:I,j=1:I,k=1:K,t=1:T], Bin); #vehicle k goes from i to j in period t
    @variable(model, y[i=1:I,k=1:K,t=1:T], Bin); #vehicle k visits node i in period t
    @variable(model, z[i=2:I,k=1:K,t=1:T] >= 0); #load carried by vehicle k when arriving at node i in period t
    @variable(model, p[i=1:I,t=1:T] >=0); # inventory at node i in time period t
    #@variable(model, r[t=1:T] >=0); #supply available at depot (node 0) at time t
    @variable(model, q[i=2:I,k=1:K,t=1:T] >=0); # Quantity delivered to customer i by vehicle k at start of period t
    #@variable(model, za >=0); #safety stock
    #@variable(model, f[r=1:10000,i=2:I,t=1:T], Bin);

    @objective(model, Min,sum(h*p[i,t] for i in 2:I, t in 1:T) + sum(shippingCost[i,j]*x[i,j,k,t] for i in 1:I,j in 1:I,k in 1:K,t in 1:T));

    #_______________ pure VRP

    # Constraint 4: A customer got visit by not more than one truck in one period:
    @constraint(model, [i=2:I,t=1:T], sum(y[i,k,t] for k in 1:K) <= 1);

    # Constraint 5: Vehicle capacity
    @constraint(model, [k=1:K,t=1:T], sum(demand[i,t]*y[i,k,t] for i in 1:I) <= Captruck);

    # Constraint 5_1 : Inflow: if vehicle k visits node i, inflow of vehicle k should be 1, 0, otherwise
    #                  Outflow: if vehicle k visits node i, outflow of vehicle k should be 1, 0, otherwise
    #part 1
    @constraint(model, [i=1:I,k=1:K,t=1:T], sum(x[i,j,k,t] for j in 1:I) == sum(x[j,i,k,t] for j in 1:I));
    #part 2
    @constraint(model, [i=1:I,k=1:K,t=1:T], sum(x[i,j,k,t] for j in 1:I) == y[i,k,t]);

    # Constraint 6: Subtour elimination constraint : load should reduce while going from i to j
    # @constraint(model, [i=2:I,j=2:J,k=1:K,t=1:T], z[i,k,t] - z[j,k,t]>= demand[i,t] -((1-x[i,j,k,t])*sum(demand[i,t] for i in 1:I)))
    @constraint(model, [i=2:I,j=2:I,k=1:K,t=1:T], z[i,k,t] - z[j,k,t] >= demand[i,t] - ((1-x[i,j,k,t]) * sum(demand[a,t] for a in 1:I) ));
    #not sure if D is sum demand from 1:I or 2:I

    #_______________ Inventory optimization

    # Constraint 0: Initializing the first set of inventories
    #@constraint(model, [i= 1:I], I_time[i,1] == I_begin[i]); # I[i,0] from slide

    # Constraint 1: Flow balance constraints at depot in each period t:


    @constraint(model, [t=1:T-1], p[1,t] == p[1,t+1] + sum(q[i,k,t+1] for i in 2:I, k in 1:K));

    # Constraint 2: Flow balance constraints at each customer i in each period t:

    @constraint(model, [i=2:I,t=1:T], (t == 1 ? Inv_begin[i] : p[i,t-1]) + sum(q[i,k,t] for k in 1:K) == p[i,t] + demand[i,t] + za*demand_std[i,t])

    # Constraint 3: Inventory capacity constraints at each customer i in each period t:

    @constraint(model, [i=2:I,t=1:T], (t == 1 ? Inv_begin[i] : p[i,t-1]) + sum(q[i,k,t] for k in 1:K) <= Capdc[i]);

    #_______________ Linking both parts

    # Constraint 7: Linking the inventory part of the model to the VRP part:
        # Constraint 7a: Amount delivered to a customer cannot exceed the holding capacity of the customer

    @constraint(model, [i=2:I,k=1:K,t=1:T], q[i,k,t] <= Capdc[i]*y[i,k,t]);

        # Constraint 7b: Inventory delivered to a customer cannot exceed the truck capacity

    @constraint(model, [k=1:K,t=1:T], sum(q[i,k,t] for i in 2:I) <= Captruck*y[1,k,t]);

    #-------
    # SOLVE
    #-------

    optimize!(model);

    println();
    #if termination_status(model) == MOI.OPTIMAL
    println("Optimal objective value = $(objective_value(model))")
    println("   ")
    println("----------------------")
    println("   ")
    #end

    ## --------- Fill rates -------------#
    fill_rates_count = zeros(I-1,T)
    Random.seed!(0);
    for r = 1:10000
        inv = Inv_begin[2:end]
        actual_demand = demand[2:end,:] + (randn(I-1,T).* demand_std[2:end,:])
        for t=1:T
            for i=1:I-1
                delivered = 0
                for k=1:K
                    delivered += value(q[i+1,k,t])
                end
                inv[i] = inv[i] + delivered - actual_demand[i,t]
                demand_met_from_stock =  max(actual_demand[i,t] + min(inv[i],0),0)
                fill_rate = demand_met_from_stock / actual_demand[i,t]
                if fill_rate>=0.95
                    fill_rates_count[i,t] += 1
                end
            end
        end 
    end

    ## ----------- Storage utilisation -------#
    storage_utilisation = zeros(I-1, T)
    inv = Inv_begin[2:end]
    for t=1:T
        for i=2:I
            inv = value(p[i,t])
            storage_utilisation[i-1,t] += round(inv/Capdc[i],digits=2)
        end
    end
    
    ## ------------ Truck utilisation -------#
    Truck_utilisation = zeros(Float64, 3, 8)
    for t=1:T
        for k = 1:K
            for i = 2:I
                Truck_utilisation[k,t] += value(q[i,k,t])
            end
            Truck_utilisation[k,t] = round(Truck_utilisation[k,t]/Captruck,digits=1)
        end
    end

    return fill_rates_count/10000, objective_value(model),Truck_utilisation,storage_utilisation
end

optimize_vrpinv (generic function with 1 method)

### (a) Truck capacity is 10 times more than in the current situation

In [5]:
za = [1.7263] #F^-1(0.99)
objectivevalues = []
truck_utils = Matrix{Float64}[]
storage_utils = Matrix{Float64}[]
global fill_rates = ones(I-1,T)
target = 0.99
step = -0.10


while all(fill_rates .> target)
    println("Iteration: ",length(za))
    println("--------------------------")
    println("Safety factor: ",za[end])
    global fill_rates,objectivevalue,truck_util,storage_util = optimize_vrpinv(za[end],10,1) #za,truck,dc
    append!(za,za[end]+step)
    append!(objectivevalues,objectivevalue)
    push!(truck_utils,truck_util)
    push!(storage_utils,storage_util)
end

println("------------------------------")
println("Incremental search finished. Best safety-factor found: ",round(za[end-1],digits=4)," optimal safety factor between $(round(za[end-1],digits=4)) and $(round(za[end],digits=4))")
println("Optimal objective value = $(objectivevalues[end-1])")

Iteration: 1
--------------------------
Safety factor: 1.7263
Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-18
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1183 rows, 1296 columns and 5101 nonzeros
Model fingerprint: 0xdd494727
Variable types: 288 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+03]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+03]
Presolve removed 213 rows and 186 columns
Presolve time: 0.01s
Presolved: 970 rows, 1110 columns, 4299 nonzeros
Variable types: 273 continuous, 837 integer (837 binary)
Found heuristic solution: objective 11021.855219

Root relaxation: objective 2.727492e+03, 437 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  D

In [6]:
println("Storage utilisation:")
println("--------------------")
storage_utils[end-1]

Storage utilisation:
--------------------


5×8 Matrix{Float64}:
 0.56  0.37  0.19  0.0   0.62  0.4   0.19  0.0
 0.71  0.48  0.24  0.0   0.46  0.27  0.16  0.0
 0.38  0.0   0.38  0.0   0.39  0.0   0.38  0.0
 0.37  0.0   0.37  0.0   0.29  0.0   0.25  0.0
 0.2   0.0   0.77  0.56  0.36  0.25  0.17  0.0

In [7]:
println("Truck utilisation:")
println("--------------------")
truck_utils[end-1]

Truck utilisation:
--------------------


3×8 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.1  0.0  0.0  0.0
 0.1  0.0  0.1  0.0  0.0  0.0  0.0  0.0

### (b) Storage capacity is 10 times more than in the current situation

In [14]:
za = [1.7263] #F^-1(0.99)
objectivevalues = []
truck_utils = Matrix{Float64}[]
storage_utils = Matrix{Float64}[]
global fill_rates = ones(I-1,T)
target = 0.99
step = -0.10


while all(fill_rates .> target)
    println("Iteration: ",length(za))
    println("--------------------------")
    println("Safety factor: ",za[end])
    global fill_rates,objectivevalue,truck_util,storage_util = optimize_vrpinv(za[end],1,10) #za,truck,dc
    append!(za,za[end]+step)
    append!(objectivevalues,objectivevalue)
    push!(truck_utils,truck_util)
    push!(storage_utils,storage_util)
end

println("------------------------------")
println("Incremental search finished. Best safety-factor found: ",round(za[end-1],digits=4)," optimal safety factor between $(round(za[end-1],digits=4)) and $(round(za[end],digits=4))")
println("Optimal objective value = $(objectivevalues[end-1])")

Iteration: 1
--------------------------
Safety factor: 1.7263
Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-18
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1183 rows, 1296 columns and 5101 nonzeros
Model fingerprint: 0x3d01b85d
Variable types: 288 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+03]
Presolve removed 194 rows and 186 columns
Presolve time: 0.01s
Presolved: 989 rows, 1110 columns, 4398 nonzeros
Variable types: 273 continuous, 837 integer (837 binary)
Found heuristic solution: objective 4385.8412192

Root relaxation: objective 1.431478e+03, 414 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  D

In [15]:
println("------------------------------")
println("Incremental search finished. Best safety-factor found: ",round(za[end-1],digits=4)," optimal safety factor between $(round(za[end-1],digits=4)) and $(round(za[end],digits=4))")
println("Optimal objective value = $(objectivevalues[end-1])")

------------------------------
Incremental search finished. Best safety-factor found: 1.4262999999999997 optimal safety factor between 1.4262999999999997 and 1.3262999999999996
Optimal objective value = 3743.53576034043


In [9]:
println("Storage utilisation:")
println("--------------------")
storage_utils[end-1]

Storage utilisation:
--------------------


5×8 Matrix{Float64}:
 0.0   0.12  0.1   0.08  0.06  0.04  0.02  0.0
 0.14  0.12  0.09  0.07  0.05  0.03  0.02  0.0
 0.27  0.23  0.19  0.16  0.12  0.08  0.04  0.0
 0.04  0.0   0.15  0.11  0.07  0.04  0.03  0.0
 0.12  0.1   0.08  0.06  0.04  0.02  0.02  0.0

In [10]:
println("Truck utilisation:")
println("--------------------")
truck_utils[end-1]

Truck utilisation:
--------------------


3×8 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.7  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.8  0.0  0.0  0.0  0.0  0.0  0.0

### (c) Both truck and storage capacity is 10 times more than in the current situation

In [11]:
za = [1.7263] #F^-1(0.99)
objectivevalues = []
truck_utils = Matrix{Float64}[]
storage_utils = Matrix{Float64}[]
global fill_rates = ones(I-1,T)
target = 0.99
step = -0.10


while all(fill_rates .> target)
    println("Iteration: ",length(za))
    println("--------------------------")
    println("Safety factor: ",za[end])
    global fill_rates,objectivevalue,truck_util,storage_util = optimize_vrpinv(za[end],10,10) #za,truck,dc
    append!(za,za[end]+step)
    append!(objectivevalues,objectivevalue)
    push!(truck_utils,truck_util)
    push!(storage_utils,storage_util)
end

println("------------------------------")
println("Incremental search finished. Best safety-factor found: ",za[end-1]," optimal safety factor between $(za[end-1]) and $(za[end])")
println("Optimal objective value = $(objectivevalues[end-1])")

Iteration: 1
--------------------------
Safety factor: 1.7263
Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-18
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1183 rows, 1296 columns and 5101 nonzeros
Model fingerprint: 0x6d6c7cf1
Variable types: 288 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+03]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+03]
Presolve removed 192 rows and 186 columns
Presolve time: 0.02s
Presolved: 991 rows, 1110 columns, 4404 nonzeros
Variable types: 273 continuous, 837 integer (837 binary)
Found heuristic solution: objective 3357.1621734

Root relaxation: objective 9.258714e+02, 389 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  D

In [12]:
println("Storage utilisation:")
println("--------------------")
storage_utils[end-1]

Storage utilisation:
--------------------


5×8 Matrix{Float64}:
 0.12  0.11  0.09  0.07  0.06  0.04  0.02  0.0
 0.13  0.11  0.08  0.06  0.04  0.02  0.01  0.0
 0.24  0.21  0.18  0.14  0.11  0.07  0.03  0.0
 0.21  0.17  0.14  0.1   0.07  0.04  0.02  0.0
 0.11  0.09  0.07  0.05  0.03  0.02  0.02  0.0

In [13]:
println("Truck utilisation:")
println("--------------------")
truck_utils[end-1]

Truck utilisation:
--------------------


3×8 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.4  0.0  0.0  0.0  0.0  0.0  0.0  0.0